## Assignment: Gameplay with RL

- Deterministic,
-  Off Policy monte carlo 
-  Q-learning

## Part 1

### Initial Setup

In [1]:
#Import relevant libraries 
import numpy as np
np.random.seed(123)
np.set_printoptions(suppress=True)

#Define Values for parameters

#Number of rounds won to win the game
d = 3

#Probability of winning given action
ph = 0.55
pl = 0.45

#Define Cost of actions
ch = 50
cl = 10
R = 1000

#Two actions, high or low effort
low = 0
high = 1

n_states = len(range(-(d),d))+1 #+1
n_actions = 2

#print(n_states)

#State Value_funct_pol_high:

#Build the game environment 
def game_env():

    p = {}
    grid = np.arange(n_states)
    #print("grid",grid)
    it = np.nditer(grid, flags=['multi_index'])

    with it:
        while not it.finished: 
            s = it.iterindex


            p[s] = {a: [] for a in range(n_actions)}

            #Game ends when player is either three rounds behind or ahead
            is_done = lambda x: x==0 or x==(n_states-1)  #x==0 or

            #Define winning criteria
            #did_win = lambda x: x==n_states

            #Final reward is 1000 if win, 0 if lose
            reward = 1000 if is_done(s) and s==6 else 0.0  #and did_win(s)  #


            if is_done(s):
                # Probability of ending up in the next state after action, next state, reward, done
                # Dep on policy


                #print("iterindex isdone", s)
                p[s][low] = [(0.5, 0, reward, True, 0.5, 0,reward, True)]       #reward()
                p[s][high]= [(0.5, 0, reward, True, 0.5, 0,reward, True)]        #reward


            else:
                #If game not done, probabilities given state, action are

                #print("iterindex else", s)

                next_s_win = s+1
                next_s_lose= s-1

                p[s][low] = [(pl, next_s_win,  reward - cl ,is_done(s),
                              1-pl, next_s_lose, reward - cl, is_done(s))]  # change to is done if trouble :  is_done(next_s_win), is_done(next_s_lose)  #'dep on next state?'

                p[s][high] = [(ph, next_s_win, reward- ch, is_done(s),
                              1-ph, next_s_lose, reward - ch,is_done(s))]   #  'dep on next state?'

            it.iternext()

    return p


### P1.1

In [2]:
#Define the deterministic policy evaluator. Takes a policy e.g array of actions given states as input

def main_dp(policy):

    #Initial state values - 0
    state_values = np.zeros(n_states)

    #Finite game
    lam = 1.0

    #Stopping criteria
    theta = 1e-6

    iteration_counter = 1

    #Policy
    #Deterministic policies

    # Variables for probabilities of actions depending on input policy

    transition_probs=game_env()
    #print("transition_probs", transition_probs)

    #iteration
    while True:
        v_old = np.copy(state_values)
        delta = 0.0


        # Define det policy as an array which gives (action | state), alter to take this array as input
        #Change if statement under to depend on det policy array.


        #Evaluate all states
        for s in range(n_states):

            if policy[s]==1:

                pi_h = 1
                pi_l = 0

                effort = "high"

            elif policy[s] ==0:
                pi_h = 0
                pi_l = 1

                effort = "high"

            else:
                print("policy error")


            v_s = 0.0

            #Evaluate each action for each state
            for a in range(n_actions):
                #print(a)

                #Assign policy probability to specific action iteration
                if a==0:
                    pi_a=pi_l
                else:
                    pi_a=pi_h

                #Probabilities given state, action

                current_entry = transition_probs[s][a][0]
                #print("current_entry", current_entry)

                p_win_h = current_entry[0]
                next_s_win = current_entry[1]   #"s+1"
                reward_win = current_entry[2]

                p_lose_h = current_entry[4]
                next_s_lose = current_entry[5]  #"s-1"
                reward_lose = current_entry[6]

                #pi_a dep on policy, #p_sa dep on action,

                #State Value function under policy (specified)
                #Action = high/low
                v_s += pi_a * ((p_win_h * (reward_win + lam * v_old[next_s_win])) + (p_lose_h * (reward_lose+ lam * v_old[next_s_lose])))

                #print("s", s, "a", a, current_entry, v_s )
            state_values[s] = v_s
            delta = np.maximum(delta, np.abs(state_values[s]-v_old[s]))
        #if (iteration_counter-1)%10==0:
            #print('After %s iterations(s):\n' %iteration_counter, state_values)
        iteration_counter +=1
        if delta < theta:   #iteration_counter >100:       #
            break
    return state_values

#Evaluate all policies with DP

#Generate all deterministic policies

def det_policies():
    from itertools import product
    policies = [i for i in product(range(2), repeat=n_states)]

    return policies

#Optimal Deterministic Policy Search

def opt_det_policy_search():
    policies = det_policies()

    max_policy = [np.zeros(n_states), np.zeros(n_states)]

    for policy in policies:
        #print(policy)

        policy_eval = main_dp(policy)
        #print("initial state under policy", policy, policy_eval[3])

        #if policy==(1, 1, 0, 0, 0, 0, 0):
         #   print("MC threshold policy", policy_eval)

        if policy_eval[3] > max_policy[1][3]:
            max_policy = [policy, policy_eval]
            #print("new max_pol", policy, "initial state value", policy_eval[3])

    print("max_policy", max_policy[0], "initial state value", max_policy[1][3])
    
#Run optimal deterministic policy evaluation
opt_det_policy_search()


max_policy (0, 0, 0, 0, 1, 1, 0) initial state value 281.65289007766967


### Discussion 

The DP approach finds an optimal policy of  (0, 0, 0, 0, 1, 1, 0) with initial state value of 281.65289007766967. 

### P1.2

In [3]:
#Define the threshold policy, returns binary action value low(0)  or high (1)
def threshold_policy(s,s_star):

    #If above threshold value, action is high = 1
    if s <= s_star:
        return 1
    else:
        return  0

#Define behaviour policy with equiprobable actions in each state
def behaviour_pol(s):

    #Random number
    random = np.random.rand(1)
    #print("random", random)
    if random>0.5:
        return 1

    else:
        return 0


#Use behaviour_policy to generate an episode
# Play game using behaviour policy until it is done
# sample from environment
#to keep track of t, t takes start t as an input. This is the t following the last state of the previous episode

def episode_gen(start_t):
    #Initialize game environment
    total_space = game_env()

    #Initialise s to mid match, aka 3
    s = 3
    done =[]
    ep_record = []
    r = 0
    ep_returns = []

    current_s = total_space[s] #find the current state action values
    #print("current_s", current_s, current_s[0][0][0])

    #Starting t
    t = start_t

    while not done:
        #print("s is", s)

        #Check if done after this one


        done = current_s[0][0][3]
        #print("done",done)

        # Generate action under under policy s
        a = behaviour_pol(s)
        #print(a)

        game_chance = np.random.rand(1)
        #print(game_chance)

        #See what outcome the  game has
        if game_chance < current_s[a][0][0]:
            #print("win")
            rt = current_s[a][0][2]
            #print("current_s", current_s)

            '''
            if not done:
                rtplus1 = total_space[s + 1][]
            else:
                rtplus1 = 0 

            '''

            # Record state, action, return
            ep_record.append([s,a,rt,t])    #cumulative returns before?

            #Add to cumulative return
            r += rt

            #Update to next s
            s= current_s[a][0][1]

            # Update current s
            current_s = total_space[s]
            #print("current_s",s, current_s)

        else:
            #print("lose")

            '''
            #Extract return at time t
            if not done:
                rtplus1 = total_space[s + 1]
            else:
                rtplus1 = 0
            '''
            rt = current_s[a][0][6]

            # Record state, action, return
            ep_record.append([s,a,rt,t])

            # Add return
            r += rt

            # Update to next s
            s = current_s[a][0][5]

            # Update current s
            current_s = total_space[s]
            #print("current_s",s, current_s)

        #Iterate t
        t += 1

    ep_returns.append(r)
    #ep_record.append([r])
    #print("ep_record",ep_record)

    return ep_record, ep_returns

#Off-policy Monte Carlo estimation
#Base on on policy prediction

def off_pol_mc_pred():
    #Set Parameters
    episodes = 500000 # 10000 #1000 #50000
    s_star = 4
    lam = 1

    #Initialise values
    returns = np.zeros([n_states])

    #Episode observations under behavioural policy
    obs_episodes = []

    #Initialize action value list as multi index
    Q_sa = np.random.randn(n_states*n_actions).reshape([n_states,n_actions])
    #print(Q_sa)

    C_sa = np.zeros(n_states*n_actions).reshape([n_states,n_actions])

    start_t = 0

    #Loop for all episodes
    for e in range(episodes):
        
        if e%100000==0:
            print(e)
        
        episode_obs, ep_returns = episode_gen(start_t)

        obs_episodes.append(episode_obs)

        g = 0
        W = 1

        start_t += len(episode_obs)
        #print(e, "ep_length", len(episode_obs), print(ep_returns))

        pi_as = 1
        b_as = 1
        pi_as_list = []
        b_as_list = []
        i = 1

        #Loop for each step of the episode
        for element in reversed(obs_episodes[e]):
            #print("step", element)
            i += 1

            #Sum returns following state of step
            try:
                g = lam*g + element[2] 
            except:
                g = lam*g

            s = element[0]
            a = element[1]
            t = element[3]

            if s==6 or s==0:
                pi_as = 1
            else:

                pi_as =  int(a==threshold_policy(s,s_star))  #probability of action given state, neither include t  #  Test policy: 0.2 if a==0 else 0.8
                #print(s,"tp", a==threshold_policy(s,s_star))

            b_as = 0.5  # equiprobable state

            #Importance sampling weights

            # C(st,at) <- C(st, at) + W
            C_sa[s][a] += W
            #Using Weighted importance sampling  #W <- W(pi(at|st)/b(at|st)
            Q_sa[s][a] += W / C_sa[s][a] * (g - Q_sa[s][a])

            #print("ep", e, element, "g", g)
            #print("Q", s,a, Q_sa[s][a])  #
            #print("pias",pi_as, "b_as",b_as)

            W = W * pi_as / b_as  # Product,   product of all action, state probabilities following

            if W ==0:
                break

        #print("pi list", pi_as_list, "b list", b_as_list)

    print("Action-state values Q_sa", Q_sa)
    for s in range(n_states):
        print("Q under threshold policy pi", Q_sa[s][threshold_policy(s, s_star)])

off_pol_mc_pred()

0
100000
200000
300000
400000
Action-state values Q_sa [[   0.            0.        ]
 [  61.50747593    4.63774701]
 [ 105.9511095   148.91343199]
 [ 330.91676949  298.46770036]
 [ 511.79507628  559.82581247]
 [ 805.57254134  774.20591761]
 [1000.         1000.        ]]
Q under threshold policy pi 0.0
Q under threshold policy pi 4.6377470117490605
Q under threshold policy pi 148.91343198555504
Q under threshold policy pi 298.4677003558798
Q under threshold policy pi 559.8258124686042
Q under threshold policy pi 805.5725413383143
Q under threshold policy pi 1000.0


### Discussion

The off policy Monte Carlo estimation under the threshold policy. A challenge in using a random policy to evaluate 
the threshold policy, which is deterministic given its threshold, is that the threshold policy probability of many of 
the sequences drop to zero whenever an action that is not strictly prescribed by the threshold policy is taken in a certain 
state. With the current behavioural policy being equiprobable, this means the probability of each state, action
sequence being under the threshold policy is 0.5^(number of states). For each backwards iteration through an episode 
this means the probability of the last action state being in the threshold policy is 0.5, the next to last 0.5^2 etc. 
which drops exponentially as the iteration continues. This means a large number of episode observations are needed in
 order to create action value estimates for the threshold policy. It also means accuracy of the estimate could be biased
 towards the extreme values, as the states prior to either losing (s=0) or winning (s=6) will more often be used as updates
 by the importance sampling procedure.   
 


## P1.3

In [4]:
def mc_optimal_policy():
    #Set Parameters
    episodes = 500000 # 10000 #1000 #50000
    lam = 1

    #Initialise values
    returns = np.zeros([n_states])

    #Episode observations under behavioural policy
    obs_episodes = []

    #Initialize action value list as multi index
    Q_sa = np.random.randn(n_states*n_actions).reshape([n_states,n_actions])
    #print(Q_sa)

    C_sa = np.zeros(n_states*n_actions).reshape([n_states,n_actions])

    start_t = 0
    
    # Define an approx optimal policy to be improved during control procedure
    mc_opt_policy = np.random.choice(2, n_states)

    #Loop for all episodes
    for e in range(episodes):
        
        if e%100000==0:
            print(e)
        
        episode_obs, ep_returns = episode_gen(start_t)

        obs_episodes.append(episode_obs)

        g = 0
        W = 1

        start_t += len(episode_obs)
        #print(e, "ep_length", len(episode_obs), print(ep_returns))

        pi_as = 1
        b_as = 1
        pi_as_list = []
        b_as_list = []
        i = 1

        #Loop for each step of the episode
        for element in reversed(obs_episodes[e]):
            #print("step", element)
            i += 1

            #Sum returns following state of step
            g = lam*g + element[2] 
            s = element[0]
            a = element[1]
            t = element[3]


            b_as = 0.5  # equiprobable state

            #Importance sampling weights

            # C(st,at) <- C(st, at) + W
            C_sa[s][a] += W
            #Using Weighted importance sampling  #W <- W(pi(at|st)/b(at|st)
            Q_sa[s][a] += W / C_sa[s][a] * (g - Q_sa[s][a])

            #print("ep", e, element, "g", g)
            #print("Q", s,a, Q_sa[s][a])  #
            #print("pias",pi_as, "b_as",b_as)
            
            # Policy improvement: in s, ppolicy_s =  a from argmax_a(Q_sa)
            mc_opt_policy[s] = np.argmax(Q_sa[s])

            if a!= mc_opt_policy[s]:
                break
            
            W = W * 1/ b_as  # Product,   product of all action, state probabilities following
          
    print("Action-state values Q_sa", Q_sa)
    print("Optimal policy p",mc_opt_policy)


mc_optimal_policy()

0
100000
200000
300000
400000
Action-state values Q_sa [[   0.            0.        ]
 [  74.50507099   15.42365236]
 [ 146.38666149  213.23475918]
 [ 369.04064858  349.27551444]
 [ 497.50829096  566.82803294]
 [ 758.08289299  766.01757565]
 [1000.         1000.        ]]
Optimal policy p [0 0 1 0 1 1 0]


### Discussion 

The off-Policy Monte Carlo algorithm for optimal policy selection above is based on the incremental implementation of Off-Poliy MC Control in  Sutton & Barto (2018 p.111). Compared to the DP approach where transition probabilities were known, the MC algorithm finds higher state action values for the actions in the initial state ( 332.96043112 & 389.75165211) than the state value of the initial state under DP (281.65). The MC Optimal policy is also different, (0 0 0 1 0 1 0) rather than (0, 0, 0, 0, 1, 1, 0). However, these policies are quite close, with the difference being optimal actions in state won 1 and 2 being swapped. Seeing that this estimate was done without knowledge of the environment, just from sampling returns, it shows how off policy MC control methods can be effective at finding approximate policies when the environment is unknown. 

# P2 

### P2.1: Q-learning Algorithm

In [ ]:
#Q -learning algorithm 

#Epislon greedy policy from class 9.
def epsilon_greedy_policy(q_values, epsilon=0.1):  
    if np.random.binomial(1, epsilon) == 1:  
        return np.random.choice(actions)  
    else:  
        return np.random.choice([action_ for action_, value_ in enumerate(q_values) if value_ == np.max(q_values)])


#def Q_learn(): 

#Set Parameters
epsilon = 0.1
episodes =  500000
alpha = 0.1
lam = 1

#Initialise arbitrary Q
Q_sa = np.random.randn(n_states*n_actions).reshape([n_states,n_actions])

q_opt_policy = np.zeros(n_states)

#Initialise state
s = 3 

#Initialise game environment
total_space = game_env()

for e in range(episodes): 

    if e%100000==0:
        print("episode", e)

    done = False
    
    while not done:    
        current_s = total_space[s]


        #Take action, (implement eps greedy)
        a =  epsilon_greedy_policy(Q_sa[s]) #np.argmax(Q_sa[s])
        #print("a", a)

        #print(current_s, current_s[0])
        game_chance = np.random.rand(1)

        #Observe what outcome the  game has
        if game_chance < current_s[a][0][0]:

            #print("win")
            rt = current_s[a][0][2]

            #Update to next s
            s_next= current_s[a][0][1]

        else:
            #print("lose")

            rt = current_s[a][0][6]

            # Update to next s
            s_next = current_s[a][0][5]


        #Update Q_sa

        Q_sa[s][a] += alpha*(rt +lam*Q_sa[s_next][np.argmax(Q_sa[s])] - Q_sa[s][a])

        #Update optimal policy
        q_opt_policy[s] = np.argmax(Q_sa[s])

        #Update s 
        s = s_next

        done = current_s[0][0][3]
        
        if s==6 or s==0:
            break 

print("Q", Q_sa, "Q policy", q_opt_policy)
